Reference: https://github.com/karpathy/ng-video-lecture

In [83]:
# Read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("Length of dataset in characters: ", len(text))


Length of dataset in characters:  1115394


In [84]:
# Let's look at the first 1000 characters
print(text[:1000])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [85]:
# Here are all the unique characters in the file
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print('Vocabulary size:', vocab_size)



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocabulary size: 65


In [86]:
# Create a mapping from character to integer
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[ch] for ch in s] # encoder: takes a string and returns a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: takes a list of integers and returns a string

print(encode("hii there"))
print(decode(encode("hii there")))


[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [87]:
"""
Other mapping methods:
    1. Google: sentencepiece
               https://github.com/google/sentencepiece

    2. OpenAI: tiktoken
               https://github.com/openai/tiktoken

               import tiktoken
               enc = tiktoken.get_encoding("gpt2")
               enc.n_vocab
               enc.encode("hii there")
               enc.decode([71, 4178, 612])

"""


'\nOther mapping methods:\n    1. Google: sentencepiece\n               https://github.com/google/sentencepiece\n\n    2. OpenAI: tiktoken\n               https://github.com/openai/tiktoken\n\n               import tiktoken\n               enc = tiktoken.get_encoding("gpt2")\n               enc.n_vocab\n               enc.encode("hii there")\n               enc.decode([71, 4178, 612])\n\n'

In [88]:
# Let's now encode the entire text dataset and store it into a torch.Tensor
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = torch.tensor(encode(text), dtype=torch.long)
data = data.to(device)
print(data.shape, data.dtype)
print(data[:100]) # the 100 chatacters we looked at


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59], device='cuda:0')


In [89]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% of the data will be used for training, rest for validation
train_data, valid_data = data[:n], data[n:]


In [90]:
valid_data.is_cuda

True

In [91]:
block_size = 8 # the sequence length of the model
train_data[:block_size+1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58], device='cuda:0')

In [92]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

"""
Let the transformer predict next character by input 1 character.
And predict next character by input 2 characters.
And predict next character by input 3 characters.
And so on.
"""


when input is tensor([18], device='cuda:0') the target is 47
when input is tensor([18, 47], device='cuda:0') the target is 56
when input is tensor([18, 47, 56], device='cuda:0') the target is 57
when input is tensor([18, 47, 56, 57], device='cuda:0') the target is 58
when input is tensor([18, 47, 56, 57, 58], device='cuda:0') the target is 1
when input is tensor([18, 47, 56, 57, 58,  1], device='cuda:0') the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15], device='cuda:0') the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47], device='cuda:0') the target is 58


'\nLet the transformer predict next character by input 1 character.\nAnd predict next character by input 2 characters.\nAnd predict next character by input 3 characters.\nAnd so on.\n'

In [93]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel
block_size = 8 # what is the maximum context length for prediction

def get_batch(split):
    # generate a small batch of inputs x and targets y
    if split == 'train':
        data = train_data
    else:
        data = valid_data

    ix = torch.randint(data.size(0) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix]) # (batch_size, block_size)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # shift 1 position to right

    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)

print('targets:')
print(yb.shape)
print(yb)

print('------')

for b in range(batch_size): # batch_dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context} the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]], device='cuda:0')
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]], device='cuda:0')
------
when input is tensor([24], device='cuda:0') the target is 43
when input is tensor([24, 43], device='cuda:0') the target is 58
when input is tensor([24, 43, 58], device='cuda:0') the target is 5
when input is tensor([24, 43, 58,  5], device='cuda:0') the target is 57
when input is tensor([24, 43, 58,  5, 57], device='cuda:0') the target is 1
when input is tensor([24, 43, 58,  5, 57,  1], device='cuda:0') the target is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46], device='cuda:0') the target is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43], de

In [94]:
yb.is_cuda

True

In [95]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            # because the second dimension of cross_entorpy represents the channels
            # we need to reshape the logits and targets
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):

        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # become (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution
            next_tokens = torch.multinomial(probs, num_samples=1) # (B,1)
            # append smapled index to the running sequence
            idx = torch.cat([idx, next_tokens], dim=-1) # (B,T+1)

        return idx
    
model = BigramLanguageModel(vocab_size)
model = model.to(device)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)
start_idx = torch.zeros((1,1), dtype=torch.long).to(device)
print(decode(model.generate(start_idx, max_new_tokens=100)[0].tolist()))
# beacuse the model is not trained yet, the output is random
# also, we only focus on the last time step, did not consider the history 


torch.Size([32, 65])
tensor(4.8786, device='cuda:0', grad_fn=<NllLossBackward0>)

pYCXxfRkRZd
wc'wfNfT;OLlTEeC K
jxqPToTb?bXAUG:C-SGJO-33SM:C?YI3a
hs:LVXJFhXeNuwqhObxZ.tSVrddXlaSZaNe


In [96]:
# create optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)


In [97]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # change to eval mode
    for split in ['train', 'valid']:
        losses = torch.zeros(eval_interval)
        for k in range(eval_interval):
            x, y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()

    return out

In [98]:
batch_size = 32
eval_interval = 1000
max_iters = 100000

for steps in range(max_iters):

    # every once in a while evaluate the loss on train and val loss
    if steps % eval_interval == 0:
        losses = estimate_loss()
        print(f"step: {steps}, train loss: {losses['train']:.4f}, val loss: {losses['valid']:.4f}")
        
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 4.7264, val loss: 4.7253
step: 1000, train loss: 3.7314, val loss: 3.7361
step: 2000, train loss: 3.1199, val loss: 3.1272
step: 3000, train loss: 2.8025, val loss: 2.8064
step: 4000, train loss: 2.6398, val loss: 2.6450
step: 5000, train loss: 2.5631, val loss: 2.5679
step: 6000, train loss: 2.5144, val loss: 2.5313
step: 7000, train loss: 2.4924, val loss: 2.5093
step: 8000, train loss: 2.4827, val loss: 2.4979
step: 9000, train loss: 2.4690, val loss: 2.4901
step: 10000, train loss: 2.4680, val loss: 2.4888
step: 11000, train loss: 2.4627, val loss: 2.4796
step: 12000, train loss: 2.4607, val loss: 2.4822
step: 13000, train loss: 2.4578, val loss: 2.4800
step: 14000, train loss: 2.4579, val loss: 2.4817
step: 15000, train loss: 2.4561, val loss: 2.4846
step: 16000, train loss: 2.4606, val loss: 2.4809
step: 17000, train loss: 2.4532, val loss: 2.4847
step: 18000, train loss: 2.4532, val loss: 2.4842
step: 19000, train loss: 2.4536, val loss: 2.4813
step: 20000, 

In [99]:
print(decode(model.generate(start_idx, max_new_tokens=400)[0].tolist()))
# characters are not talk to each other


Wawice my.

Hastarom oroup
Yowhthetof isth ble mil ndill, ath iree sengmin lat Heriliovets, and Win nghirileranousel lind me l.
HAshe ce hiry:
Supr aisspllw y.
Hentofu n Boopetelaves
MP:

Pl, d mothakleo Windo whth eisbyo the m dourive we higend t so mower; te

AN ad nterupt f s ar igr t m:

Thiny aleronth,
Mad
Whed my o myr f-bube!
KENoby ak
Sadsal thes ghesthidin cour ay aney Iry ts I fr y ce.
J
